In [1]:
#| default_exp data.database.sqlalchemy

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#| hide
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import os
import pandas as pd 

In [6]:
from sqlalchemy import MetaData, create_engine, asc, desc, and_, or_, not_, case, extract, cast, text
from sqlalchemy.types import DateTime, Date, Time
from sqlalchemy.schema import *
from sqlalchemy.sql import func as F, Selectable, select
from sqlalchemy.dialects import registry
from sqlalchemy.engine.row import Row
from sqlalchemy.orm import Session

In [7]:
#| export
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials


In [8]:
#| hide
from google.cloud import bigquery
from dotenv import load_dotenv


In [9]:

load_dotenv("../.env")

True

In [10]:
os.environ

environ{'GSM_SKIP_SSH_AGENT_WORKAROUND': 'true',
        'LC_ALL': 'en_US.UTF-8',
        'WINDOWPATH': '2',
        'PATH': '/home/x/.pyenv/versions/miniforge3-latest/envs/prc/bin:/home/x/.pyenv/versions/miniforge3-latest/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin',
        'CONDA_PREFIX': '/home/x/.pyenv/versions/miniforge3-latest/envs/prc',
        'LIBVIRT_DEFAULT_URI': 'qemu:///system',
        'LOGNAME': 'x',
        'XDG_CONFIG_DIRS': '/etc/xdg/xdg-cinnamon:/etc/xdg',
        'XAUTHORITY': '/run/user/1000/gdm/Xauthority',
        'XDG_SESSION_ID': '3',
        'CONDA_SHLVL': '1',
        'XMODIFIERS': '@im=ibus',
        'GIO_LAUNCHED_DESKTOP_FILE_PID': '215017',
        'GIO_LAUNCHED_DESKTOP_FILE': '/home/x/Desktop/pycharm.desktop',
        'QT_ACCESSIBILITY': '1',
        'CONDA_DEFAULT_ENV': 'prc',
        'XDG_SEAT': 'seat0',
        'XDG_VTNR': '2',
        'QTWEBENGINE_DICTIONARIES_PATH': '/usr/shar

In [11]:
# os.getenv('GC_QUOTE_API_CREDENTIALS')
# os.getenv('HTTP_PROXY')
# os.getenv('HTTPS_PROXY')

In [12]:
# config = dotenv_values("../.env")
# key = config.get('GC_QUOTE_API_CREDENTIALS')
key = os.getenv('GC_QUOTE_API_CREDENTIALS')

In [13]:
credential = service_account.Credentials.from_service_account_file("../" + key)


In [14]:

#os.environ['HTTP_PROXY'] = "http://127.0.0.1:20171"
#os.environ['HTTPS_PROXY'] = "http://127.0.0.1:20171"
#os.environ['http_proxy'] = "http://127.0.0.1:20171"
#os.environ['https_proxy'] = "http://127.0.0.1:20171"



In [15]:
registry.register('bigquery', 'sqlalchemy_bigquery', 'BigQueryDialect')
engine = create_engine('bigquery://quote-api-365206',
                       credentials_path='../' + key,)


In [16]:
metadata = MetaData()
def get_table(project_name: str, dataset_name: str, table_name: str)-> Table:
    table = Table(f'{project_name}.{dataset_name}.{table_name}', metadata, autoload_with=engine)
    return table

In [17]:
# table = get_table('quote-api-365206', 'report', 'ride_dispatch_v')

In [18]:
# query = select(table).limit(20).where(table.c.ride_stat == 'Accepted').order_by(asc(table.c.ride_id))
# df = pd.read_sql(query, engine)
# df


In [19]:
# table = get_table('elife-data-warehouse-prod', 'ods', 'ride_ride')

In [20]:
# table

In [21]:
# query = select(table).where(table.c.airport_code3 == 'JFK').limit(20)

In [22]:
# df = pd.read_sql(query, engine)
# df


# find all tables in the database

In [23]:
ride_trip_t = get_table('elife-data-warehouse-prod','ods', 'ride_trip').alias()
ride_dispatch_t = get_table('elife-data-warehouse-prod','ods', 'ride_dispatch').alias()
ride_enum_t = get_table('elife-data-warehouse-prod','ods', 'ride_enum').alias()
ride_ride_t = get_table('elife-data-warehouse-prod','ods', 'ride_ride').alias()
ride_partner_tran_t = get_table('elife-data-warehouse-prod','ods', 'ride_partner_tran').alias()
ride_partner_t = get_table('elife-data-warehouse-prod','ods', 'ride_partner').alias()
dim_place_t = get_table('elife-data-warehouse-prod','dim', 'dim_place').alias()
ride_vehicle_class_t = get_table('elife-data-warehouse-prod','ods', 'ride_vehicle_class').alias()

In [24]:
auction_ride_t = get_table('elife-data-warehouse-prod','ods', 'ride_auction_ride').alias()
auction_fleet_t = get_table('elife-data-warehouse-prod','ods', 'ride_auction_fleet').alias()
ride_fleet_t = get_table('elife-data-warehouse-prod','ods', 'ride_fleet').alias()



In [25]:
trip_type_id_t = select(ride_dispatch_t.c.ride_id, ride_trip_t.c.trip_type.label('trip_type_id'))
trip_type_id_t = trip_type_id_t.select_from(ride_dispatch_t
                                      .join(ride_trip_t, ride_dispatch_t.c.ride_id == ride_trip_t.c.ride_id))
trip_type_id_t = trip_type_id_t.alias()
                  

In [26]:
# trip_type_id_t = trip_type_id_t.limit(10)
# df = pd.read_sql(trip_type_id_t, engine)
# df

In [27]:
trip_type_t = select(trip_type_id_t.c.ride_id, trip_type_id_t.c.trip_type_id, ride_enum_t.c.name.label('trip_type'))
trip_type_t = trip_type_t.select_from(trip_type_id_t
                                      .join(ride_enum_t, trip_type_id_t.c.trip_type_id == ride_enum_t.c.id, isouter=True))
trip_type_t = trip_type_t.alias()


In [28]:
ride_status_t = select(ride_ride_t.c.id.label('ride_id'), ride_ride_t.c.stat.label('ride_status_id'), ride_enum_t.c.name.label('ride_status'))
ride_status_t = ride_status_t.select_from(ride_ride_t
                                          .join(ride_enum_t, ride_ride_t.c.stat == ride_enum_t.c.id, isouter=True))
ride_status_t = ride_status_t.alias()
# ride_status_t = ride_status_t.limit(10)
# df = pd.read_sql(ride_status_t, engine)
# df

## Traffic peak time exclusion
source https://www.quora.com/What-is-the-trickiest-time-of-the-day-to-drive-in-Orlando

In [29]:
#ride_datetime_tz_t = select(ride_datetime_t.c.ride_id,
#                         # F.timestamp(ride_datetime_t.c.from_datetime).label('from_datetime_utc'),
#                         ride_datetime_t.c.from_datetime,
#                         ride_datetime_t.c.from_datetime_str,
#                         cast(ride_datetime_t.c.from_date_str, Date).label('from_date'),
#                         cast(ride_datetime_t.c.from_time_str, Time).label('from_time'),
#                         # ride_datetime_t.c.from_date_str,
#                         # ride_datetime_t.c.from_time_str,
#                         ride_datetime_t.c.utc,
#                         ride_datetime_t.c.tz,
#                         ride_datetime_t.c.day_of_week,
#                         F.datetime(F.timestamp_seconds(ride_datetime_t.c.utc)).label('from_datetime_utc'),
#                         #F.format_timestamp('%Y-%m-%d %H:%M:%S', F.timestamp_seconds(ride_datetime_t.c.utc), ride_datetime_t.c.tz).label('datetime_with_new_timezone')
#                         # text("TIMESTAMP(ride_datetime_t.c.from_datetime) AT TIME ZONE 'UTC' AT TIME ZONE ride_datetime_t.c.tz").label('datetime_with_new_timezone') 
#                         #F.datetime(F.timestamp_seconds(ride_datetime_t.c.utc), F.timezone(ride_datetime_t.c.tz)).label('from_datetime_local'),
#                         # cast(ride_datetime_t.c.from_datetime_str, DateTime).label('from_datetime_tz'),
#                         )
#ride_datetime_tz_t = ride_datetime_tz_t.select_from(ride_datetime_t) #.join(ride_ride_t, ride_ride_t.c.id == ride_datetime_t.c.ride_id))
## print(ride_non_peak_t.compile(compile_kwargs={"literal_binds": True}))
#ride_datetime_tz_t = ride_datetime_tz_t.alias()
## ride_datetime_tz_t = ride_datetime_tz_t.limit(100)
## df = pd.read_sql(ride_datetime_tz_t,engine)
## df.dtypes
## df

In [30]:
ride_datetime_t = select(ride_ride_t.c.id.label('ride_id'), 
                         F.substring(ride_ride_t.c.from_time_str, 1, 10).label('from_date_str'),
                         F.concat(F.substring(ride_ride_t.c.from_time_str, 12, 16),':00').label('from_time_fix_str'),
                         F.concat(F.substring(ride_ride_t.c.from_time_str, 1, 10), 
                                  ' ', 
                                  F.substring(ride_ride_t.c.from_time_str, 12, 16),
                                  ':00'
                                  ).label('from_datetime_fix_str'),
                         extract(
                             'DAYOFWEEK',
                             cast(F.concat(F.substring(ride_ride_t.c.from_time_str, 1, 10),
                                       ' ',
                                       F.substring(ride_ride_t.c.from_time_str, 12, 16),
                                       ':00'
                                       ),
                              DateTime)
                         ).label('day_of_week_local'),
                         extract(
                             'DAYOFWEEK',
                             cast(F.timestamp_seconds(ride_ride_t.c.from_utc), 
                                  DateTime)
                         ).label('day_of_week_utc'),
                         F.datetime(F.timestamp_seconds(ride_ride_t.c.from_utc)).label('from_datetime_utc'),
                         ride_ride_t.c.from_timezone_str,
                         # F.current_date(ride_ride_t.c.from_timezone_str).label('current_date'),
                         #cast(ride_ride_t.c.from_time_str,DateTime).label('from_datetime'),
                         #cast(F.concat(F.substring(ride_ride_t.c.from_time_str, 1, 10), 'T', F.substring(ride_ride_t.c.from_time_str, 12, 16)),DateTime).label('from_datetime'),
                         #cast(ride_ride_t.c.from_time_str, DateTime).label('from_time_dt'),
                         # F.timezone(ride_ride_t.c.from_timezone_str, cast(ride_ride_t.c.from_time_str, DateTime).label('from_time_dt')),
                         )                  
ride_datetime_t = ride_datetime_t.select_from(ride_ride_t)
ride_datetime_t = ride_datetime_t.alias()
# ride_datetime_t = ride_datetime_t.limit(100)
# df = pd.read_sql(ride_datetime_t, engine)

In [31]:
#df

In [32]:
from workalendar.usa import Florida 
import numpy as np
cal_florida = Florida()
exclude_dates_str = [str(d[0]) for d in cal_florida.holidays(2024)]
exclude_dates_str

['2024-01-01',
 '2024-01-15',
 '2024-05-27',
 '2024-07-04',
 '2024-09-02',
 '2024-11-11',
 '2024-11-28',
 '2024-11-29',
 '2024-12-25']

In [33]:
peak_time_str = [('07:00:00', '09:00:00'), ('16:00:00', '19:00:00')]
night_time_str = [('22:00:00', '6:00:00')]  # Shanghai, US usually no overtime extra fees New York 8pm ~ 6am
ind = []
td = []
for pt in peak_time_str:
    ind.append(pd.DatetimeIndex(pt))
ind
for i in ind:
    i[1]-i[0]

[DatetimeIndex(['2024-09-14 07:00:00', '2024-09-14 09:00:00'], dtype='datetime64[ns]', freq=None),
 DatetimeIndex(['2024-09-14 16:00:00', '2024-09-14 19:00:00'], dtype='datetime64[ns]', freq=None)]

Timedelta('0 days 02:00:00')

Timedelta('0 days 03:00:00')

In [34]:
peak_time = []
for pt in peak_time_str:
    peak_time.append(pd.date_range(pt[0], pt[1], freq='h'))
for pt in peak_time:
    print(pt, pt.time)

DatetimeIndex(['2024-09-14 07:00:00', '2024-09-14 08:00:00',
               '2024-09-14 09:00:00'],
              dtype='datetime64[ns]', freq='h') [datetime.time(7, 0) datetime.time(8, 0) datetime.time(9, 0)]
DatetimeIndex(['2024-09-14 16:00:00', '2024-09-14 17:00:00',
               '2024-09-14 18:00:00', '2024-09-14 19:00:00'],
              dtype='datetime64[ns]', freq='h') [datetime.time(16, 0) datetime.time(17, 0) datetime.time(18, 0)
 datetime.time(19, 0)]


In [35]:
peak_time_str

[('07:00:00', '09:00:00'), ('16:00:00', '19:00:00')]

In [36]:
#ride_non_peak_t = select(ride_datetime_t).where(
#            # not_(ride_datetime_tz_t.c.day_of_week.in_([1, 7])),
#            # not_(ride_datetime_tz_t.c.from_datetime_str.in_(exclude_dates)),
#            and_(
#                # not_(ride_datetime_t.c.day_of_week.in_([1, 7])),
#                not_(ride_datetime_t.c.from_datetime_str.in_(exclude_dates_str)),
#                not_(
#                    and_(
#                        # text(f"TIME({ride_datetime_tz_t.c.from_time}) > TIME({peak_time_str[0][0]}) AND TIME({ride_datetime_tz_t.c.from_time}) < TIME({peak_time_str[0][1]})"),
#                        # text(f"TIME({ride_datetime_tz_t.c.from_time}) > TIME({peak_time_str[1][0]}) AND TIME({ride_datetime_tz_t.c.from_time}) < TIME({peak_time_str[1][1]})"),
#                        cast(ride_datetime_t.c.from_time, Time) > cast(peak_time_str[0][0],Time),
#                        cast(ride_datetime_t.c.from_time, Time) < cast(peak_time_str[0][1],Time),
#                    )
#                ),
#                not_(
#                    and_(
#                        cast(ride_datetime_t.c.from_time, Time) > cast(peak_time_str[1][0],Time),
#                        cast(ride_datetime_t.c.from_time, Time) < cast(peak_time_str[1][1],Time),
#                    )
#                )
#            )
#        )               
#ride_non_peak_t = ride_non_peak_t.select_from(ride_datetime_t)
## ride_non_peak_t = ride_non_peak_t.alias()
#ride_non_peak_t = ride_non_peak_t.limit(1000)
#df = pd.read_sql(ride_non_peak_t,engine)
#df.head()

In [37]:
# df

In [38]:
exclude_dates = [d[0] for d in cal_florida.holidays(2024)]
# exclude_dates
# res = df['from_datetime_utc'].apply(lambda x: x in exclude_dates)

In [39]:
# np.any(res.apply(lambda x: x in exclude_dates))
# res[0]=True
# res
# np.any(res)


In [40]:
dispatch_status_t = select(ride_dispatch_t.c.id.label('ride_id'), ride_dispatch_t.c.stat.label('dispatch_status_id'), ride_enum_t.c.name.label('distpatch_status'))
dispatch_status_t = dispatch_status_t.select_from(ride_dispatch_t
                                          .join(ride_enum_t, ride_dispatch_t.c.stat == ride_enum_t.c.id, isouter=True))
dispatch_status_t = dispatch_status_t.alias()
#dispatch_status_t = dispatch_status_t.limit(10)
#df = pd.read_sql(dispatch_status_t, engine)
#df

In [41]:

expr = case(
    (ride_dispatch_t.c.to_fleet_id == auction_fleet_t.c.fleet_id, 'auction'), 
    else_='dispatch',
)


In [42]:
auction_type_t = select(
    auction_ride_t.c.ride_id.label('ride_id'),
    auction_ride_t.c.auction_id.label('auction_id'),
    auction_fleet_t.c.fleet_id.label('auction_fleet_id'),
    ride_dispatch_t.c.to_fleet_id.label('dispatch_fleet_id'),
    ride_fleet_t.c.name.label('fleet'),
    case(
        (ride_dispatch_t.c.to_fleet_id == auction_fleet_t.c.fleet_id, 'auction'),
        else_ = 'dispatch',
    ).label('dispatch_type')
)
auction_type_t = auction_type_t.select_from(auction_ride_t
                                            .join(auction_fleet_t, auction_ride_t.c.auction_id == auction_fleet_t.c.auction_id, isouter=True)
                                            .join(ride_dispatch_t, auction_ride_t.c.ride_id == ride_dispatch_t.c.ride_id, isouter=True)
                                            .join(ride_fleet_t, ride_dispatch_t.c.to_fleet_id == ride_fleet_t.c.id, isouter=True))
auction_type_t = auction_type_t.alias()
# auction_type_t = auction_type_t.limit(100)
# df = pd.read_sql(auction_type_t, engine)
# df

In [43]:
dispatch_type_t = (select(
    ride_dispatch_t.c.id.label('ride_id'),
    ride_dispatch_t.c.to_fleet_id.label('dispatch_fleet_id'),
    auction_ride_t.c.auction_id.label('auction_id'),
    auction_fleet_t.c.fleet_id.label('auction_fleet_id'),
    case(
        (ride_dispatch_t.c.to_fleet_id == auction_fleet_t.c.fleet_id, 'auction'), 
        else_ = 'dispatch',
    ).label('dispatch_type'),
    ride_fleet_t.c.name.label('fleet')
))
# .where(
#     ride_dispatch_t.c.to_fleet_id == auction_fleet_t.c.fleet_id
# ))
dispatch_type_t = dispatch_type_t.select_from(ride_dispatch_t
                                          .join(auction_ride_t, ride_dispatch_t.c.ride_id == auction_ride_t.c.ride_id, isouter=True)
                                          .join(auction_fleet_t, auction_ride_t.c.auction_id == auction_fleet_t.c.auction_id, isouter=True)
                                          .join(ride_fleet_t, ride_dispatch_t.c.to_fleet_id == ride_fleet_t.c.id, isouter=True))
dispatch_type_t = dispatch_type_t.alias()
# dispatch_type_t = dispatch_type_t.limit(100)
# df = pd.read_sql(dispatch_type_t, engine)
# df

In [44]:
partner_id_t = select(ride_ride_t.c.id.label('ride_id'), ride_partner_tran_t.c.partner_id.label('partner_id'))
partner_id_t = partner_id_t.select_from(ride_ride_t
                                        .join(ride_partner_tran_t, ride_ride_t.c.partner_tran_id == ride_partner_tran_t.c.id, isouter=True))
partner_id_t = partner_id_t.alias()
#partner_id_t = partner_id_t.limit(10)
#df = pd.read_sql(partner_id_t, engine)
#df

In [45]:
partner_t = select(partner_id_t.c.ride_id, partner_id_t.c.partner_id, ride_partner_t.c.name.label('partner'))
partner_t = partner_t.select_from(partner_id_t
                                  .join(ride_partner_t, partner_id_t.c.partner_id == ride_partner_t.c.id, isouter=True))
partner_t = partner_t.alias()
# partner_t = partner_t.limit(10)
# df = pd.read_sql(partner_t,engine)
# df

In [46]:
from_place_t = select(ride_ride_t.c.id.label('ride_id'), 
                      ride_ride_t.c.from_place_id.label('start_place_id'), 
                      dim_place_t.c.name.label('start_place'), 
                      dim_place_t.c.lng.label('lng'), 
                      dim_place_t.c.lat.label('ltt'),
                      )
from_place_t = from_place_t.select_from(ride_ride_t
                                        .join(dim_place_t, ride_ride_t.c.from_place_id == dim_place_t.c.id, isouter=True))
from_place_t = from_place_t.alias()
#from_place_t = from_place_t.limit(10)
#df = pd.read_sql(from_place_t, engine)
#df

In [47]:
to_place_t = select(ride_ride_t.c.id.label('ride_id'), 
                      ride_ride_t.c.to_place_id.label('end_place_id'), 
                      dim_place_t.c.name.label('end_place'), 
                      dim_place_t.c.lng.label('lng'), 
                      dim_place_t.c.lat.label('ltt'))
to_place_t = to_place_t.select_from(ride_ride_t
                                        .join(dim_place_t, ride_ride_t.c.to_place_id == dim_place_t.c.id, isouter=True))
to_place_t = to_place_t.alias()
#to_place_t = to_place_t.limit(10)
#df = pd.read_sql(to_place_t, engine)
#df

In [48]:
vehicle_class_t = select(ride_ride_t.c.id.label('ride_id'),
                         ride_ride_t.c.vehicle_class_id.label('vehicle_class_id'), 
                         ride_vehicle_class_t.c.name.label('vehicle_class'))
vehicle_class_t = vehicle_class_t.select_from(ride_ride_t
                                              .join(ride_vehicle_class_t, ride_ride_t.c.vehicle_class_id == ride_vehicle_class_t.c.id, isouter=True))
vehicle_class_t = vehicle_class_t.alias()
#vehicle_class_t = vehicle_class_t.limit(10)
#df = pd.read_sql(vehicle_class_t, engine)
#df

In [49]:
from workalendar.usa import Florida

In [50]:
cal_orlando = Florida()
data_from_date = '2024-01-01'


In [51]:
exclude_dates = [d[0] for d in cal_orlando.holidays(2024)]
exclude_dates

[datetime.date(2024, 1, 1),
 datetime.date(2024, 1, 15),
 datetime.date(2024, 5, 27),
 datetime.date(2024, 7, 4),
 datetime.date(2024, 9, 2),
 datetime.date(2024, 11, 11),
 datetime.date(2024, 11, 28),
 datetime.date(2024, 11, 29),
 datetime.date(2024, 12, 25)]

In [52]:
price_training_t = select(ride_ride_t.c.id.label('ride_id'),
                          ride_ride_t.c.trip_count,
                          ride_ride_t.c.from_utc,
                          ride_ride_t.c.from_time_str,
                          ride_ride_t.c.from_timezone_str,
                          ride_ride_t.c.to_time_str,
                          ride_ride_t.c.to_timezone_str,
                          ride_ride_t.c.passenger_count,
                          ride_ride_t.c.luggage_count,
                          ride_ride_t.c.children_count,
                          ride_ride_t.c.infant_count,
                          ride_ride_t.c.distance,
                          ride_ride_t.c.duration,
                          ride_dispatch_t.c.id.label('dispatch_id'),
                          ride_dispatch_t.c.trip_no,
                          ride_dispatch_t.c.amount.label('dispatch_amount'),
                          ride_dispatch_t.c.currency.label('dispatch_currency'),
                          ride_datetime_t.c.from_date_str,
                          ride_datetime_t.c.from_time_fix_str,
                          ride_datetime_t.c.from_datetime_fix_str,
                          # ride_datetime_t.c.day_of_week,
                          # text("EXTRACT(DAYOFWEEK FROM TIMESTAMP_SECONDS(ride_ride_t.from_utc))").label('day_of_week'),
                          trip_type_t.c.trip_type_id,
                          trip_type_t.c.trip_type,
                          ride_status_t.c.ride_status_id,
                          ride_status_t.c.ride_status,
                          dispatch_status_t.c.dispatch_status_id,
                          dispatch_status_t.c.distpatch_status,
                          dispatch_type_t.c.dispatch_type,
                          ride_fleet_t.c.name.label('fleet'),
                          partner_t.c.partner_id,
                          partner_t.c.partner,
                          from_place_t.c.start_place_id,
                          from_place_t.c.start_place,
                          from_place_t.c.lng.label('start_lng'),
                          from_place_t.c.ltt.label('start_ltt'),
                          to_place_t.c.end_place_id,
                          to_place_t.c.end_place,
                          to_place_t.c.lng.label('end_lng'),
                          to_place_t.c.ltt.label('end_ltt'),
                          vehicle_class_t.c.vehicle_class_id,
                          vehicle_class_t.c.vehicle_class,
                          ).where(
                              and_(
                                  ride_ride_t.c.from_time_str > data_from_date,
                                  ride_dispatch_t.c.currency == 'USD',
                                  vehicle_class_t.c.vehicle_class== 'MPV-5',
                                  or_ (dispatch_type_t.c.dispatch_type == 'auction',
                                       dispatch_type_t.c.dispatch_type == 'dispatch'),
                                  F.lower(from_place_t.c.start_place).like('%orlando%'),
                                  F.lower(to_place_t.c.end_place).like('%orlando%'),
                                  
                                  # not_(ride_datetime_t.c.day_of_week.in_([1, 7])),
                                  # not_(ride_ride_t.c.from_time_str.in_(exclude_dates)),
                              )
                          )

In [53]:

price_training_t = price_training_t.select_from(ride_ride_t
                                                .join(ride_dispatch_t, ride_ride_t.c.id == ride_dispatch_t.c.ride_id)
                                                .join(ride_datetime_t, ride_ride_t.c.id == ride_datetime_t.c.ride_id)
                                                .join(trip_type_t, ride_ride_t.c.id == trip_type_t.c.ride_id, isouter=True)
                                                .join(ride_status_t, ride_ride_t.c.id == ride_status_t.c.ride_id, isouter=True)
                                                .join(dispatch_status_t, ride_ride_t.c.id == dispatch_status_t.c.ride_id, isouter=True)
                                                .join(dispatch_type_t, ride_ride_t.c.id == dispatch_type_t.c.ride_id, isouter=True)
                                                .join(partner_t, ride_ride_t.c.id == partner_t.c.ride_id, isouter=True)
                                                .join(from_place_t, ride_ride_t.c.id == from_place_t.c.ride_id, isouter=True)
                                                .join(to_place_t, ride_ride_t.c.id == to_place_t.c.ride_id, isouter=True)
                                                .join(vehicle_class_t, ride_ride_t.c.id == vehicle_class_t.c.ride_id, isouter=True)
                                                .join(ride_fleet_t, ride_dispatch_t.c.to_fleet_id == ride_fleet_t.c.id, isouter=True)
                                                )
                                                                                                                                    
#price_training_t = price_training_t.where(ride_ride_t.c.from_time_str > '2024-01-01')
price_training_t = price_training_t.limit(1000)


In [54]:
df = pd.read_sql(price_training_t, engine)
df

,ride_id,trip_count,from_utc,from_time_str,from_timezone_str,to_time_str,to_timezone_str,passenger_count,luggage_count,children_count,...,start_place_id,start_place,start_lng,start_ltt,end_place_id,end_place,end_lng,end_ltt,vehicle_class_id,vehicle_class
0,3237873,1,1720310100,2024-07-06 19:55,America/New_York,None,None,4,4.0,NaN,...,1164523,"Orlando International Airport (MCO), 1 Jeff Fu...",-81.308332,28.429425,1164524,"409 FL-527, Orlando, FL 32801, USA",-81.376616,28.547888,3,MPV-5
1,3237873,1,1720310100,2024-07-06 19:55,America/New_York,None,None,4,4.0,NaN,...,1164523,"Orlando International Airport (MCO), 1 Jeff Fu...",-81.308332,28.429425,1164524,"409 FL-527, Orlando, FL 32801, USA",-81.376616,28.547888,3,MPV-5
2,3237873,1,1720310100,2024-07-06 19:55,America/New_York,None,None,4,4.0,NaN,...,1164523,"Orlando International Airport (MCO), 1 Jeff Fu...",-81.308332,28.429425,1164524,"409 FL-527, Orlando, FL 32801, USA",-81.376616,28.547888,3,MPV-5
3,3237873,1,1720310100,2024-07-06 19:55,America/New_York,None,None,4,4.0,NaN,...,1164523,"Orlando International Airport (MCO), 1 Jeff Fu...",-81.308332,28.429425,1164524,"409 FL-527, Orlando, FL 32801, USA",-81.376616,28.547888,3,MPV-5
4,3122071,1,1705025100,2024-01-11 21:05,America/New_York,None,None,4,4.0,NaN,...,946274,"Orlando International Airport (MCO), 1 Jeff Fu...",-81.308332,28.429425,946275,"Universal's Loews Portofino Bay Hotel, 5601 Un...",-81.460235,28.480653,3,MPV-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,2806494,1,1711605720,2024-03-28 02:02,Eastern Daylight Time,None,None,2,2.0,0.0,...,381714,"Orlando International Airport (MCO), 1 Jeff Fu...",-81.308332,28.429425,381715,"Universal's Loews Sapphire Falls Resort, 6601 ...",-81.470804,28.466518,3,MPV-5
645,2858428,1,1704978000,2024-01-11 08:00,America/New_York,None,None,3,3.0,NaN,...,474099,"Bluegreen Vacations Fountains, Ascend Resort C...",-81.478587,28.389856,474100,"Orlando International Airport (MCO), 1 Jeff Fu...",-81.308332,28.429425,3,MPV-5
646,2858428,1,1704978000,2024-01-11 08:00,America/New_York,None,None,3,3.0,NaN,...,474099,"Bluegreen Vacations Fountains, Ascend Resort C...",-81.478587,28.389856,474100,"Orlando International Airport (MCO), 1 Jeff Fu...",-81.308332,28.429425,3,MPV-5
647,2858428,1,1704978000,2024-01-11 08:00,America/New_York,None,None,3,3.0,NaN,...,474099,"Bluegreen Vacations Fountains, Ascend Resort C...",-81.478587,28.389856,474100,"Orlando International Airport (MCO), 1 Jeff Fu...",-81.308332,28.429425,3,MPV-5


In [58]:
sqlite_eng = create_engine('sqlite:///../data/price_training_from_gbq_raw.db', echo=False)

In [59]:
df.to_sql('price_training_orlando_mpv5', sqlite_eng, if_exists='replace')

649

In [115]:
#orlando_airport = pd.read_csv('../../data/orlando_all_output.csv')
#orlando_airport.head()
#orlando_airport.dtypes
#orlando_airport.to_sql('orlando_airport', sqlite_eng, if_exists='append')



In [55]:
df_training = df.loc[:,['ride_id', 'trip_type', 'trip_no', 'trip_count', 'ride_status', 'partner', 'fleet', 
                'from_time_str', 'from_timezone_str', 
                'start_place', 'end_place', 
                'distance', 'duration', 'vehicle_class', 
                'passenger_count', 'luggage_count', 
                'dispatch_amount', 'dispatch_currency']]
df_training['cent_price_per_km'] = df_training['dispatch_amount'] / df_training['distance']*100.0
df_training

,ride_id,trip_type,trip_no,trip_count,ride_status,partner,fleet,from_time_str,from_timezone_str,start_place,end_place,distance,duration,vehicle_class,passenger_count,luggage_count,dispatch_amount,dispatch_currency,cent_price_per_km
0,3237873,point2point,0,1,Pending,Booking,Yorvis Hernández,2024-07-06 19:55,America/New_York,"Orlando International Airport (MCO), 1 Jeff Fu...","409 FL-527, Orlando, FL 32801, USA",20571,1454,MPV-5,4,4.0,30.66,USD,0.149045
1,3237873,point2point,0,1,Pending,Booking,Yorvis Hernández,2024-07-06 19:55,America/New_York,"Orlando International Airport (MCO), 1 Jeff Fu...","409 FL-527, Orlando, FL 32801, USA",20571,1454,MPV-5,4,4.0,30.66,USD,0.149045
2,3237873,point2point,-1,1,Pending,Booking,Elife,2024-07-06 19:55,America/New_York,"Orlando International Airport (MCO), 1 Jeff Fu...","409 FL-527, Orlando, FL 32801, USA",20571,1454,MPV-5,4,4.0,40.88,USD,0.198726
3,3237873,point2point,-1,1,Pending,Booking,Elife,2024-07-06 19:55,America/New_York,"Orlando International Airport (MCO), 1 Jeff Fu...","409 FL-527, Orlando, FL 32801, USA",20571,1454,MPV-5,4,4.0,40.88,USD,0.198726
4,3122071,point2point,0,1,Pending,Booking,Lumasini Transportation,2024-01-11 21:05,America/New_York,"Orlando International Airport (MCO), 1 Jeff Fu...","Universal's Loews Portofino Bay Hotel, 5601 Un...",29638,1449,MPV-5,4,4.0,33.15,USD,0.111850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,2806494,None,0,1,Pending,Booking,xiaorong wu,2024-03-28 02:02,Eastern Daylight Time,"Orlando International Airport (MCO), 1 Jeff Fu...","Universal's Loews Sapphire Falls Resort, 6601 ...",27519,1293,MPV-5,2,2.0,45.00,USD,0.163523
645,2858428,None,0,1,Pending,Booking,CARLOS RODRIGO BELTRAN LOPEZ,2024-01-11 08:00,America/New_York,"Bluegreen Vacations Fountains, Ascend Resort C...","Orlando International Airport (MCO), 1 Jeff Fu...",25063,1440,MPV-5,3,3.0,35.00,USD,0.139648
646,2858428,None,0,1,Pending,Booking,CARLOS RODRIGO BELTRAN LOPEZ,2024-01-11 08:00,America/New_York,"Bluegreen Vacations Fountains, Ascend Resort C...","Orlando International Airport (MCO), 1 Jeff Fu...",25063,1440,MPV-5,3,3.0,35.00,USD,0.139648
647,2858428,None,-1,1,Pending,Booking,Elife,2024-01-11 08:00,America/New_York,"Bluegreen Vacations Fountains, Ascend Resort C...","Orlando International Airport (MCO), 1 Jeff Fu...",25063,1440,MPV-5,3,3.0,40.30,USD,0.160795


In [47]:

df_training['cent_per_km'] = df_training['dispatch_amount'] / df_training['distance'] * 100.0
df_training


,ride_id,trip_type,trip_no,trip_count,ride_status,partner,fleet,from_time_str,from_timezone_str,start_place,end_place,distance,duration,vehicle_class,passenger_count,luggage_count,dispatch_amount,dispatch_currency,price_per_km,cent_per_km
0,3231310,point2point,-1,1,Pending,Booking,Elife,2024-03-01 19:54,America/New_York,"Orlando International Airport (MCO), 1 Jeff Fu...",Clarion Inn & Suites Across From Universal Orl...,28177,1284,MPV-4,4,4,36.99,USD,0.131277,0.131277
1,3231310,point2point,-1,1,Pending,Booking,Elife,2024-03-01 19:54,America/New_York,"Orlando International Airport (MCO), 1 Jeff Fu...",Clarion Inn & Suites Across From Universal Orl...,28177,1284,MPV-4,4,4,36.99,USD,0.131277,0.131277
2,3231310,point2point,-1,1,Pending,Booking,Elife,2024-03-01 19:54,America/New_York,"Orlando International Airport (MCO), 1 Jeff Fu...",Clarion Inn & Suites Across From Universal Orl...,28177,1284,MPV-4,4,4,36.99,USD,0.131277,0.131277
3,3259057,point2point,0,1,Pending,Booking,DMS Car Service,2024-03-17 17:25,America/New_York,"Orlando International Airport (MCO), 1 Jeff Fu...","Universal's Aventura Hotel, 6725 Adventure Wy,...",27370,1403,Sedan,3,3,33.00,USD,0.120570,0.120570
4,3259057,point2point,0,1,Pending,Booking,DMS Car Service,2024-03-17 17:25,America/New_York,"Orlando International Airport (MCO), 1 Jeff Fu...","Universal's Aventura Hotel, 6725 Adventure Wy,...",27370,1403,Sedan,3,3,33.00,USD,0.120570,0.120570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,3015680,point2point,0,2,Pending,CityAirportTaxis,wagner Valladao de Araujo Filho,2024-01-09 18:20,America/New_York,Orlando Airport,Orlando Marriott Lake Mary,60193,2470,Sedan,1,1,47.50,USD,0.078913,0.078913
196,3015680,point2point,0,2,Pending,CityAirportTaxis,wagner Valladao de Araujo Filho,2024-01-09 18:20,America/New_York,Orlando Airport,Orlando Marriott Lake Mary,60193,2470,Sedan,1,1,47.50,USD,0.078913,0.078913
197,3015680,point2point,0,2,Pending,CityAirportTaxis,wagner Valladao de Araujo Filho,2024-01-09 18:20,America/New_York,Orlando Airport,Orlando Marriott Lake Mary,60193,2470,Sedan,1,1,47.50,USD,0.078913,0.078913
198,3015680,point2point,0,2,Pending,CityAirportTaxis,wagner Valladao de Araujo Filho,2024-01-09 18:20,America/New_York,Orlando Airport,Orlando Marriott Lake Mary,60193,2470,Sedan,1,1,47.50,USD,0.078913,0.078913


In [33]:
#| hide
import nbdev; nbdev.nbdev_export()